`pip install SQLAlchemy` --- general-purpose library for interacting with SQL Databases through python.

`pip install psycopg2-binary` --- postgres-specific "driver"

What you need to connect to any database server / database in the world:
1. host
2. port
3. username
4. password
5. database name

In [1]:
from sqlalchemy import create_engine

In [2]:
HOST = 'convolutional-curry.cxk5vhnfj8ju.eu-central-1.rds.amazonaws.com'
PORT = '5432'
USERNAME = 'postgres'
PASSWORD = 'titanic99'
DB = 'postgres'

conn_string = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}"
conn = create_engine(conn_string)

conn.execute("CREATE TABLE IF NOT EXISTS test (hi TEXT);")

conn.execute("INSERT INTO test (hi) VALUES ('hello convolutional curry!');")

result = conn.execute("SELECT * FROM test;")

In [3]:
# result.first()
#the result of a query returns a reference to the query response, which you have to "unpack" yourself
#this is because it's possible that the query response is too large for python to handle.

In [4]:
# result.fetchall()
#result is a "generator-like" object. Once you start pulling data from it, it gets removed.

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(result.fetchall())

---

Connecting to another database -- pretty simple; just replace the values of the variables!

In [7]:
HOST = 'convolutional-curry.cxk5vhnfj8ju.eu-central-1.rds.amazonaws.com'
PORT = '5432'
USERNAME = 'postgres'
PASSWORD = 'titanic99'
DB = 'darias_northwind'

conn_string_northwind = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}"
conn_northwind = create_engine(conn_string_northwind)

In [8]:
conn_northwind.execute('SELECT COUNT(*) FROM orders;').fetchall()

[(830,)]

In [9]:
df = pd.read_sql_table('orders', conn_northwind)

---

In [10]:
df.to_csv('darias_orders_table.csv')

In [11]:
df.to_sql('new_orders', conn, if_exists='replace')

In [12]:
query = '''
SELECT customerid, orderid 
FROM orders WHERE shipvia > 10;
'''


pd.read_sql_query(query, conn_northwind)

,customerid,orderid


If you want to explore SQLAlchemy further and write SQL Queries more elegantly without having to write messy strings, you need to use the Object Relational Mapper (ORM). 11.7 of the course material.

---

You can also set `echo=True` to see what SQL queries that SQLAlchemy is running under the hood for you.
- also, if you want to specify datatypes, you can also import things like VARCHAR, TEXT, and INTEGER from sqlalchemy.

In [13]:
HOST = 'convolutional-curry.cxk5vhnfj8ju.eu-central-1.rds.amazonaws.com'
PORT = '5432'
USERNAME = 'postgres'
PASSWORD = 'titanic99'
DB = 'postgres'

conn_string = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}"
conn = create_engine(conn_string, echo=True)

In [14]:
df = pd.read_csv('darias_orders_table.csv', index_col=0)

In [15]:
from sqlalchemy.types import VARCHAR
# from sqlalchemy.dialects import ...

In [16]:
df.to_sql('new_orders', conn, if_exists='replace', dtype={'customerid':VARCHAR(length=10)})

2021-06-10 11:04:56,557 INFO sqlalchemy.engine.base.Engine select version()
2021-06-10 11:04:56,559 INFO sqlalchemy.engine.base.Engine {}
2021-06-10 11:04:56,594 INFO sqlalchemy.engine.base.Engine select current_schema()
2021-06-10 11:04:56,595 INFO sqlalchemy.engine.base.Engine {}
2021-06-10 11:04:56,626 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-06-10 11:04:56,627 INFO sqlalchemy.engine.base.Engine {}
2021-06-10 11:04:56,643 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-06-10 11:04:56,643 INFO sqlalchemy.engine.base.Engine {}
2021-06-10 11:04:56,659 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2021-06-10 11:04:56,659 INFO sqlalchemy.engine.base.Engine {}
2021-06-10 11:04:56,693 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20